In [ ]:
%matplotlib inline
import yfinance as yf
import pandas as pd
import matplotlib.pyplot as plt
#import seaborn; seaborn.set()
#from pandas_datareader import data as pdr
from bs4 import BeautifulSoup

# Задание 1.

Вы - менеджер по продажам в нефтехимической компании. Вы ведете несколько ключевых клиентов, которые закупают продукты вашей компании. Ваша компания запускает новый продукт - "Волшебный белый порошок" (ВБП), который по своим свойствам похож на тот, что уже есть на рынке. Ваши клиенты предпочитают формульный подход к ценообразованию. Вам нужно предложить клиентам формулу расчета цены на ВБП. В зависимости от объемов, которые покупают клиенты, им может быть предложена скидка.

### Дополнительные вводные:

Так как продукт Б новый, возможно стоит предложить дисконт от цены Продукта А. Продукт производиться из нефти. Один из известных способов производства предполагает что из 16 баррелей нефти сделают одну тонну ВБП, а затраты на производство в районе 400 евро.

In [ ]:
# Из данных есть:
# Цены на продукт А, который уже есть на рынке Европы
# ----- октябрь 2018 - 
# ----- ноябрь 2018 - 
# ----- февраль 2019 -

# Котировки нефти, курс eur/usd
dbc = pd.read_excel('cur_oil.xlsx')

# Затраты на производство
PRODUCTION_COST = 400 # (EUR)

# Расходы на логистику
EU_LOGISTIC_COST_EUR = 30 # в Европу в евро
CN_LOGISTIC_COST_USD = 130 # в Китай в долларах

# * Справочная информация по клиентам(объемы, локации, комментарии) 
customers = {
    'Monty':{
        'location':'EU',
        'volumes':200,
        'comment':'moving_average'
    },
    
    'Triangle':{
        'location':'CN',
        'volumes': 30,
        'comment': 'monthly'
    },
    'Stone':{
        'location':'EU',
        'volumes': 150,
        'comment': 'moving_average'
    },
    'Poly':{
        'location':'EU',
        'volumes': 70,
        'comment': 'monthly'
    }
}
# Скидки
discounts = {'up to 100': 0.01, # 1%
             'up to 300': 0.05, # 5%
             '300 plus': 0.1}   #10%


### Что нужно сделать?

1. Проанализировать имеющиеся данные.
2. Определить базовую формулу цены на условиях FCA (цена на заводе). То есть как бы выглядела цена на исторических данных.
3. Отобразить на графике.
4. Сделать расчет возможной цены по формуле для каждого из клиентов на условиях DDP (цена с доставкой). Записать все в один эксель файл, на разных листах. Каждый лист - название клиента.
5. Предложить формулу цены каждому из клиентов на условиях DDP (цена с доставкой).
- Создать директорию «для клиентов» и в ней сложить  файлы с расчетами.
- Каждый клиент - отдельный файл (любой из docx, xlsx, pptx, pdf)  с именем клиента… 

### В результате должно быть 3 файла:
1. Ноутбук с расчетами и графиками. 
2. Скрипт, который делает расчет цены для каждого клиента в одном файле
2. Скрипт, который делает расчет цены для каждого клиента в отельном файле

#### Задание 1.
1. Проанализировать имеющиеся данные.
2. Определить базовую формулу цены на условиях FCA (цена на заводе). То есть как бы выглядела цена на исторических данных.
3. Отобразить на графике.
4. Сделать расчет возможной цены по формуле для каждого из клиентов на условиях DDP (цена с доставкой). Записать все в один эксель файл, на разных листах. Каждый лист - название клиента.
5. Предложить формулу цены каждому из клиентов на условиях DDP (цена с доставкой).
 - Создать директорию «для клиентов» и в ней сложить файлы с расчетами.
 - Каждый клиент - отдельный файл (любой из docx, xlsx, pptx, pdf) с именем клиента…

In [ ]:
#1.Проанализировать имеющиеся данные.
#2.Определить базовую формулу цены на условиях FCA (цена на заводе). То есть как бы выглядела цена на исторических
#данных.

class MagicWhitePowder:
        """Формула расчета цены на продукт"""
        
        def __init__(self):
            self.PRODUCTION_COST = 400
            self.EU_LOGISTIC_COST_EUR = 30 # в Европу в евро
            self.CN_LOGISTIC_COST_USD = 130 # в Китай в долларах
        
        def logistic_cost (self, region, EUR_USD):
            assert region in ["EU", "CN"], f"Не известный регион {region}"
            if region == 'EU':
                cost = self.EU_LOGISTIC_COST_EUR
            elif  region == 'CN':
                cost = self.CN_LOGISTIC_COST_USD / EUR_USD
            return cost
        
        def fca(self, oil_price, EUR_USD):
            """Расчет цены на заводе"""
            return  16 * oil_price / EUR_USD + self.PRODUCTION_COST    
        
        def ddp(self, oil_price, EUR_USD, region):
            """Расчет цены c доставкой для клиента"""
            return self.fca(oil_price, EUR_USD) +  self.logistic_cost(region, EUR_USD)

In [ ]:
#3.Отобразить на графике.
#4.Сделать расчет возможной цены по формуле для каждого из клиентов на условиях DDP (цена с доставкой). 
#Записать все в один эксель файл, на разных листах. Каждый лист - название клиента.

dbc = pd.read_excel('cur_oil.xlsx')
dbc.dropna(inplace=True)
mwp = MagicWhitePowder()

writer = pd.ExcelWriter('MWP_prices.xlsx', engine = 'xlsxwriter')

for key, value in customers.items():
    region = value['location']
    dbc['FCA'] = mwp.fca(dbc['OIL'], dbc['EURUSD=X'])
    dbc['DDP'] = mwp.ddp(dbc['OIL'], dbc['EURUSD=X'], region)
    
    dbc.to_excel(writer, sheet_name = key)

writer.save()

#Отобразить цену FCA на графике.
dbc.plot(x='Date', y='FCA')
dbc.head(5)

### Расчет цены для подхода Moving Avarage

FCA = MA(Oil_price, window=5) * MA(EURUSD, window=5) * 16 + PRODUCTION_COST 

DDP = FCA + LOGISTIC_COST

In [ ]:
#5. Предложить формулу цены каждому из клиентов на условиях DDP (цена с доставкой).
# - Создать директорию «для клиентов» и в ней сложить файлы с расчетами.
# - Каждый клиент - отдельный файл (любой из docx, xlsx, pptx, pdf) с именем клиента…
dbc_ma = dbc.copy()

dbc_ma['EURUSD=X'] = dbc_ma['EURUSD=X'].rolling(window=5).mean()
dbc_ma['OIL']= dbc_ma['OIL'].rolling(window=5).mean()

dbc_mtha = dbc.copy()
dbc_mtha = dbc_mtha.set_index('Date').groupby(pd.Grouper(freq='M')).mean()  

for key, value in customers.items():
    writer = pd.ExcelWriter(f'{key}.xlsx', engine = 'xlsxwriter')
    region = value['location']
    meaning = value['comment']
    
    if meaning == 'moving_average':
        dbc_ma['FCA'] = mwp.fca(dbc_ma['OIL'], dbc_ma['EURUSD=X'])
        dbc_ma['DDP'] = mwp.ddp(dbc_ma['OIL'], dbc_ma['EURUSD=X'], region)
        dbc_ma.to_excel(writer, sheet_name = key)
    elif meaning == 'monthly':
        dbc_mtha['FCA'] = mwp.fca(dbc_mtha['OIL'], dbc_mtha['EURUSD=X'])
        dbc_mtha['DDP'] = mwp.ddp(dbc_mtha['OIL'], dbc_mtha['EURUSD=X'], region)
        dbc_mtha.to_excel(writer, sheet_name = key)

    writer.save()
#writer.close()

# Задание 2. Продолжение

Клиенты согласились на ваше предложение. 
Вам нужно сделать написать скрипт, с помощью которого можно автоматом рассчитывать цены для всех клиентов одновременно. 
Важный момент: данные для расчета скрипт должен подтягивать сам.

В результате должно быть 3 файла
1. Ноутбук с расчетами и графиками (чтобы данные подтягивались туда)
2. Скрипт, который делает расчет цены для клиента (чтобы данные подтягивались туда сами)

In [ ]:
eurusd = yf.Ticker("EURUSD=X").history(period="1mo")
oil = yf.Ticker("CL=F").history(period="1mo")
dbc_ma = pd.DataFrame([])
dbc_ma['FCA'] = mwp.fca(oil['Close'], eurusd['Close'])
dbc_ma.dropna()
dbc_ma.plot()

prices = {'FCA': {}, 'DDP': {}}

for key, value in customers.items():
    region = value['location']
    prices['FCA'][key]= mwp.fca(oil.tail(1)['Close'][0], eurusd.tail(1)['Close'][0])
    prices['DDP'][key]= mwp.ddp(oil.tail(1)['Close'][0], eurusd.tail(1)['Close'][0], region)

df_prices = pd.DataFrame(prices)
df_prices


# Задание 3. Back to Russia

К вам пришел новый клиент из России. Который использует продукт А и готов регулярно закупать ВБП для того, чтобы не быть привязанным к одному поставщику. Клиент готов забирать ВБП с завода самостоятельно.
1. На основе тех данных, которые уже есть предложите цену новому клиенту. 
2. Подготовьте скрипт, который будет использовать курс публикуемый на <a href='http://www.cbr.ru/currency_base/dynamics/'> сайте ЦБ РФ</a>.


In [ ]:
from urllib.request import urlopen

mwp = MagicWhitePowder()

webpage = urlopen('http://www.cbr.ru/currency_base/daily/').read().decode('utf-8')
soup = BeautifulSoup(webpage)
#print(webpage)
for tr in soup.findAll('tr'):
    if len(tr.findAll('td')) < 5:
        continue
    tdl = tr.findAll('td')
    #print(tdl)
    #for td in tr.findAll('td'):
    if tdl[1].text == "EUR":
        #print(tdl[4].text)
        EURRUB = float(tdl[4].text.replace(',', '.'))

eurusd = yf.Ticker("EURUSD=X").history(period="1mo")
oil = yf.Ticker("CL=F").history(period="1mo")

prices_FCA = mwp.fca(oil.tail(1)['Close'][0], eurusd.tail(1)['Close'][0]) * EURRUB
prices_FCA

# Задание 4. Check it

* Подумайте, как можно проверить корректность работы алгоритма? 
* Какие возможны ошибки?
* Придумайте тесты для проверки возможных ошибок алгоритма
* Интергрируйте их в свои скрипты в виде исключений


In [ ]:
mwp = MagicWhitePowder()

assert mwp.fca(10, 1) == 560, "Не правильная формула FCA"
assert mwp.ddp(10, 1, "EU") == 590, "Не правильная формула DDP для региона 'EU' "
assert mwp.ddp(10, 1, "CN") == 690, "Не правильная формула DDP для региона 'CN' "
assert mwp.ddp(10, 1, "RU") == 690

# Задание 5. Поделись с ближним

Вы решили пойти в отпуск на время вашего отсутствия вас будет заменять коллега. Он будет заниматься рассчетом цен для клиентов. Чтобы ему было проще вам нужно завернуть ваши скрипты в модуль, который будет запускать у себя коллега.

* Создайте модуль, который можно запускать в ноутбуке подкладывая в ту же директорию
* Хорошенько задокументируйте ВСЁ, но помните о PEP8

In [ ]:
import mwp
mwp.customers
m = mwp.MagicWhitePowder()
m.fca(oil_price=10, EUR_USD=1)


# Задание 6. Поделись пакетом

Ваши коллеги узнали, что вы быстро рассчитываете цены для формульных контрактов, и решили использовать такую же практику для своих продуктов и клиентов. Вам нужно создать пакет, чтобы его было проще устанавливать и использовать.

* Создайте отдельный пакет, который можно устанавливать с помощью pip 
* В пакете должны быть реализованы функции для рассчета цен по формульным контрактам
* Выложите пакет в репозиторий

In [20]:
!pip install mwp-1.1.tar.gz
from mwp import mwp

Processing ./mwp-1.1.tar.gz
DEPRECATION: Source distribution is being reinstalled despite an installed package having the same name and version as the installed package. pip 21.1 will remove support for this functionality. A possible replacement is use --force-reinstall. You can find discussion regarding this at https://github.com/pypa/pip/issues/8711.
  Created wheel for mwp: filename=mwp-1.1-py3-none-any.whl size=2502 sha256=29dd7b8128f1980c3ae0d62cb53bd7135c06a4c8831046f47f6d6741cdfbafbc
  Stored in directory: /Users/annavelikobratova/Library/Caches/pip/wheels/99/4f/dd/9f83ef7f214412f9f09169acea435ba9f47ce0a3ce4e37fba8
Successfully built mwp
  Attempting uninstall: mwp
    Found existing installation: mwp 1.1
    Uninstalling mwp-1.1:
      Successfully uninstalled mwp-1.1


In [24]:
m = mwp.MagicWhitePowder()
m.fca(10, 1)

mwp.dbc

,Date,EURUSD=X,OIL
0,2018-01-01,1.2005,NaN
1,2018-01-02,1.2012,59.15
2,2018-01-03,1.2063,59.15
3,2018-01-04,1.2010,59.15
4,2018-01-05,1.2069,60.44
...,...,...,...
341,2019-04-23,1.1259,67.72
342,2019-04-24,1.1224,67.14
343,2019-04-25,1.1153,66.37
344,2019-04-26,1.1137,64.27
